In [1]:
# Garante que o Colab utilizará o TensorFlow 2.x
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import itertools
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import datasets
from tensorflow.keras import optimizers

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import TensorBoard


In [6]:
data = np.load('ORL_faces.npz')

In [7]:
X_train = data['trainX']
X_train = np.array(X_train, dtype=np.float32)/255

X_test = data['testX']
X_test = np.array(X_test, dtype=np.float32)/255

y_train = data['trainY']
y_test = data['testY']

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.05, random_state=42)

In [9]:
im_rows = 112
im_cols = 92
im_shape = (im_rows, im_cols, 1)

1. **Reformate** os dados de treinamento, validação e teste no formato adequado para a rede neural.

In [10]:
X_train = X_train.reshape(X_train.shape[0], *im_shape)
X_test = X_test.reshape(X_test.shape[0], *im_shape)
X_valid = X_valid.reshape(X_valid.shape[0], *im_shape)

2. **Construa** a rede neural convolucional objetivando o resultado da métrica apresentado no final deste notebook.

In [11]:
cnn_model= Sequential([
    Conv2D(filters=36, kernel_size=7, activation='relu', input_shape= im_shape),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=54, kernel_size=5, activation='relu', input_shape= im_shape),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(2024, activation='relu'),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    #20 is the number of outputs
    Dense(20, activation='softmax')  
])

3. **Compile** o modelo com os seguintes parâmetros:
* **Loss:** sparse_categorical_entropy;
* **Optimizer:** Adam;
* **Metrics:** accuracy.

In [12]:
cnn_model.compile(
    loss='sparse_categorical_crossentropy',#'categorical_crossentropy',
    optimizer=optimizers.Adam(lr=0.0001),
    metrics=['accuracy']
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


4. **Ajuste** o modelo aos dados de treinamento considerando os seguintes parâmetros:
* batch_size = 512
* verbose = 2

In [13]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 106, 86, 36)       1800      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 53, 43, 36)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 49, 39, 54)        48654     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 19, 54)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 24624)             0         
                                                                 
 dense (Dense)               (None, 2024)              4

In [14]:
history=cnn_model.fit(
    np.array(X_train), np.array(y_train), batch_size=512,
    epochs=250, verbose=2,
    validation_data=(np.array(X_valid),np.array(y_valid)),
)

Epoch 1/250
1/1 - 20s - loss: 3.0104 - accuracy: 0.0526 - val_loss: 2.9826 - val_accuracy: 0.0000e+00 - 20s/epoch - 20s/step
Epoch 2/250
1/1 - 8s - loss: 3.0127 - accuracy: 0.0658 - val_loss: 2.9855 - val_accuracy: 0.0000e+00 - 8s/epoch - 8s/step
Epoch 3/250
1/1 - 8s - loss: 3.0008 - accuracy: 0.0746 - val_loss: 2.9983 - val_accuracy: 0.0000e+00 - 8s/epoch - 8s/step
Epoch 4/250
1/1 - 7s - loss: 2.9843 - accuracy: 0.0658 - val_loss: 3.0097 - val_accuracy: 0.0000e+00 - 7s/epoch - 7s/step
Epoch 5/250
1/1 - 8s - loss: 3.0120 - accuracy: 0.0395 - val_loss: 3.0134 - val_accuracy: 0.0000e+00 - 8s/epoch - 8s/step
Epoch 6/250
1/1 - 7s - loss: 3.0093 - accuracy: 0.1009 - val_loss: 3.0134 - val_accuracy: 0.0000e+00 - 7s/epoch - 7s/step
Epoch 7/250
1/1 - 8s - loss: 2.9731 - accuracy: 0.0833 - val_loss: 3.0131 - val_accuracy: 0.0000e+00 - 8s/epoch - 8s/step
Epoch 8/250
1/1 - 7s - loss: 2.9806 - accuracy: 0.0746 - val_loss: 3.0087 - val_accuracy: 0.0000e+00 - 7s/epoch - 7s/step
Epoch 9/250
1/1 - 7s 

In [15]:
scor = cnn_model.evaluate( np.array(X_test),  np.array(y_test), verbose=0)

print('Test los {:.4f}'.format(scor[0]))
print('Test acc {:.4f}'.format(scor[1]))

Test los 0.2997
Test acc 0.9438
